# Example usage of package

First import the package, and pathlib which is required to handle files.


In [ ]:
import lidar
print(f"package version: {lidar.__version__}")
from pathlib import Path

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

Ignore the warning, which comes from the rospy package.

If you are useing VS code and the devcontainer then everything is setup for you (recommended). Otherwise you can choose to use the Docker file in the .devcontainer folder or make your own virtual environment.

If you geta ModuleNotFoundError for lidar make sure that you actuall installed the package. This can be done with 

$ pip install -e .   

in the folder where setup.py is.

## Reading a ROS .bag file into the lidar.Dataset

In [ ]:
testbag = Path().cwd().parent.joinpath("tests/testdata/test.bag")
testbag

In [ ]:
testset = lidar.Dataset(testbag,topic="/os1_cloud_node/points",keep_zeros=False)

This reads the bagfile into the Dataset.
Dataset only reads frames from the bagfile if needed, in order to save memory and make it possible to work which huge bagfiles.

In [ ]:
print(testset)

In [ ]:
len(testset)

In order to see whats availble use "tab" to see the availble properties and methods. Alterantivly, use help(), dir(), and the documentation.
Also shift tab is nice inside jupyter lab.

In [ ]:
help(testset)

In [ ]:
dir(testset)

In [ ]:
testset.topics_in_bag

You can also work diretly with the bag if needed.

In [ ]:
type(testset.bag)

In [ ]:
testset.end_time

# Work with Frames

They are based on pandas dataframes and pyntcloud.
This was necessary since, no pointcloud library currently support to store automotive lidar data which consists of more than just y,x,z and maybe R,G,B

First grab the first frame in the dataset:

In [ ]:
testframe = testset[0]

In [ ]:
print(testframe)

Note that the number of points can vary, since all zero elements are deltede on import (see option keep_zero in the dataset)

In [ ]:
len(testframe)

## Plotting
Tip: move the mouse over the points to get detailed information

In [ ]:
testframe.plot_interactive(color="intensity", point_size=0.5)

This plot uses plotly as the backend, which can be rather time consuming. 
WARNING: delte the output cells with plotly plots, they make the file very big.

Aternativly you can use:

In [ ]:
testframe.plot_interactive(backend="pyntcloud")

## Working with pointcouds
The frame consists maily of the properties "data" and "points".

In [ ]:
testframe.data

So data contains everything as a pandas dataframe. With all its power.

In [ ]:
testframe.data.describe()

In [ ]:
testframe.data.hist();

Now a closer look a the points. 

In [ ]:
testframe.points

So its a Pyntcloud object https://pyntcloud.readthedocs.io/en/latest/PyntCloud.html which in turn is also based on Dataframes with many methods for pointclouds.
In order to access the dataframe use this:

You can also work with the pointcloud with open3d

In [ ]:
open3d_points = testframe.get_open3d_points()

In [ ]:
open3d_points.get_max_bound()

## Pointcloud processing with build in methods
Although you can do a lot with just data and points, on its own the Frame object has methods build in for processing, which in turn return a frame object. The use the power of dataframes, pyntcloud and open3d.


In [ ]:
newframe = testframe.limit("x",-5,5).limit("intensity",400,1000)

In [ ]:
newframe.describe()

So this is now a smaller Frame with x ranging from -5 to  5, and with intenisties above 400. Processing steps can be chained together sicne the return a new Frame object.

# Plane segmenation, Clustering and overlaying several plots
Please not that not all processing methods are demonstrated here. For more infor please refer to the html documenation of the Frame class.

In [ ]:
plane = newframe.plane_segmentation(distance_threshold= 0.01,ransac_n= 3,num_iterations= 50)
print(len(plane))

In [ ]:
clusters = newframe.get_cluster(eps=0.5, min_points= 10)
cluster1 = newframe.take_cluster(0,clusters)
cluster2 = newframe.take_cluster(1,clusters)
print(len(cluster1))
print(len(cluster2))

In [ ]:
newframe.plot_overlay({"Plane": plane,"Cluster 1": cluster1,"Cluster 2": cluster2})

# Frame from a las file.
Since the frame object is based on pyntcloud we can ready any file format supported by pyntcloud.

In [ ]:
lasframe = lidar.from_file(Path('/workspaces/lidar/tests/testdata/diamond.las'))

In [ ]:
lasframe.plot_interactive()